In [ ]:
from google.colab import drive
drive.mount('/content/drive')
debug = False

Mounted at /content/drive


In [ ]:
import os
os.getcwd()

'/content'

In [ ]:
os.chdir("/content/drive/MyDrive/thesis/2_MODELING")

In [ ]:
import pandas as pd
train = pd.read_parquet("../files/processed/paired_datasets/train.parquet")



In [ ]:
train=train[['vacant_id', 'pos_candidate_id', 'neg_candidate_id','pos_candidate_full_text', 'neg_candidate_full_text','neg_vacant_full_text']]

In [ ]:
import random
def pick_random_chunks(job_text, pos_cand_text, neg_cand_text):
    def sample(s):
        l = len(s)
        if l <= 512:
            return s
        i = random.randint(0, l - 512)
        return s[i:i+512]

    return {
        "anchor":   sample(job_text),
        "positive": sample(pos_cand_text),
        "negative": sample(neg_cand_text),
    }
frames=[]
# then:
for i in range(4):
    tr = train[['vacant_id', 'pos_candidate_id', 'neg_candidate_id',
                'pos_candidate_full_text', 'neg_candidate_full_text',
                'neg_vacant_full_text']].copy()

    tr[['anchor', 'positive', 'negative']] = tr.apply(
        lambda r: pick_random_chunks(
            r['neg_vacant_full_text'],          # job
            r['pos_candidate_full_text'],       # pos cv
            r['neg_candidate_full_text'],       # neg cv
        ),
        axis=1,
        result_type="expand"
    )

    tr = tr[['vacant_id', 'pos_candidate_id', 'neg_candidate_id',
             'anchor', 'positive', 'negative']]
    frames.append(tr)


In [ ]:
train = pd.concat(frames)

In [ ]:
train

In [ ]:
val = pd.read_parquet("../files/processed/paired_datasets/val.parquet")
val=val[['vacant_id', 'pos_candidate_id', 'neg_candidate_id','pos_candidate_full_text', 'neg_candidate_full_text','neg_vacant_full_text']]


In [ ]:
val

In [ ]:
frames=[]
for i in range(4):
    tr = val[['vacant_id', 'pos_candidate_id', 'neg_candidate_id',
                'pos_candidate_full_text', 'neg_candidate_full_text',
                'neg_vacant_full_text']].copy()

    tr[['anchor', 'positive', 'negative']] = tr.apply(
        lambda r: pick_random_chunks(
            r['neg_vacant_full_text'],          # job
            r['pos_candidate_full_text'],       # pos cv
            r['neg_candidate_full_text'],       # neg cv
        ),
        axis=1,
        result_type="expand"
    )

    tr = tr[['vacant_id', 'pos_candidate_id', 'neg_candidate_id',
             'anchor', 'positive', 'negative']]
    frames.append(tr)


In [ ]:
val = pd.concat(frames)
val

In [ ]:
cols ={ "pos_chunks":"positive", 	"neg_chunks":"negative", 	"job_chunks":"anchor"}
train = train.rename(columns = cols)
val = val.rename(columns = cols)


In [ ]:
from qa_model.loader import make_triplet_collate_fn, TripletDataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModel



In [1]:
from datasets import load_dataset
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
    SentenceTransformerModelCardData,
)
from sentence_transformers.losses import MultipleNegativesRankingLoss
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers.evaluation import TripletEvaluator
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

model = SentenceTransformer(
    "sentence-transformers/multi-qa-mpnet-base-cos-v1",
    device=device,          # <--- forces GPU if available
)

Using device: cpu


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from datasets import Dataset, DatasetDict

train_hf = Dataset.from_pandas(
    train[['anchor', 'positive', 'negative']],
    preserve_index=False
)

val_hf = Dataset.from_pandas(
    val[['anchor', 'positive', 'negative']],
    preserve_index=False
)


dataset = DatasetDict({
    "train": train_hf,
    "dev": val_hf
})

dataset


DatasetDict({
    train: Dataset({
        features: ['anchor', 'positive', 'negative'],
        num_rows: 1087128
    })
    dev: Dataset({
        features: ['anchor', 'positive', 'negative'],
        num_rows: 563380
    })
})

In [ ]:
train_dataset = dataset["train"]
eval_dataset = dataset["dev"]


In [ ]:

loss = MultipleNegativesRankingLoss(model)

args = SentenceTransformerTrainingArguments(
    output_dir="./first_try/mpnet-base-all-nli-triplet",
    num_train_epochs=2,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    learning_rate=2e-5,
    warmup_ratio=0.1,
    fp16=False,  
    bf16=True, 
    batch_sampler=BatchSamplers.NO_DUPLICATES,  
    save_total_limit=2,
    logging_steps=10000,
)




In [ ]:

dev_evaluator = TripletEvaluator(
    anchors=eval_dataset["anchor"],
    positives=eval_dataset["positive"],
    negatives=eval_dataset["negative"],
    name="all-nli-dev",
)

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=loss,
    evaluator=dev_evaluator,
)
trainer.train()

Step,Training Loss
10000,2.388200
20000,2.937700


RuntimeError: [enforce fail at inline_container.cc:664] . unexpected pos 568273856 vs 568273748

In [ ]:


# (Optional) Evaluate the trained model on the test set
test_evaluator = TripletEvaluator(
    anchors=eval_dataset["anchor"],
    positives=eval_dataset["positive"],
    negatives=eval_dataset["negative"],
)
test_evaluator(model)

#{'cosine_accuracy': 0.589992446899}

In [ ]:
# 8. Save the trained model
model.save_pretrained("./with_base/2_epochs")
